In [19]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom

from datetime import date
from datetime import time
from datetime import timedelta

from pandas import DataFrame
from pandas import concat
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras.model import Sequential
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

from numpy import array
#from keras.layers import Dropout
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
from datetime import datetime
from numba import jit

In [3]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
#from treeviz import tree_print  # to print decision tree

In [4]:
file_export = 'export2012.csv'
data = pd.read_csv(file_export)
df_sorted= data.sort_values(by=['case', 'startTime'])
data = df_sorted.copy().loc[:,['case', 'event', 'startTime', 'completeTime']]

In [5]:
data = pd.read_csv(file_export)

In [6]:
data_sorted = data.sort_values(by=['case', 'event','completeTime'])

In [7]:
# data["next_task"] = calculator_nb(data)

data_iterrated = data.copy()
# for i in data_iterrated.iterrows():
#     if (i[0]+1 >= len(data)):
#         break
#     data["next_task"] = data.at[i[0]+1, 'case'] if (data.at[i[0] + 1, "case"] == i[1]["case"]) else 'null'

@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        # print(idx)
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]
        # res[idx] = event[idx + 1]  if (case[idx + 1] != case[idx])
        # else:
        #     res[idx] = 0

        idx+=1
    return res


# print(data_iterrated['case'].array)
data_iterrated["next_task"] = calculator_nb(data_iterrated['case'].values, data_iterrated["event"].values)

<ipython-input-7-aaad0d85d9a0>:9: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': cannot determine Numba type of <class 'type'>

File "<ipython-input-7-aaad0d85d9a0>", line 11:
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    ^

  @jit(parallel = True)
<ipython-input-7-aaad0d85d9a0>:9: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': cannot determine Numba type of <class 'type'>

File "<ipython-input-7-aaad0d85d9a0>", line 11:
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    ^

  @jit(parallel = True)
C:\ProgramData\Anaconda3\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceobj=True, but has

In [8]:
data_train, data_test = train_test_split(data_iterrated,test_size=0.2, shuffle=False)

In [9]:
data_iterrated.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED


In [10]:
list_of_events = data_iterrated["event"].unique()
event_to_num = {}
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1

In [11]:
event_to_num['None'] = 23

Naive predictor. First, event predictor is constructed.

In [12]:
#For each event, counts how many times another event occurs after it.
#Indexed based on list_of_events
count = {}
# initialise the count dictionary with zeros, otherwise it does not work
for event in list_of_events:
    #for k in range( len(event_to_num) ):
        count[str(event)] = [0 for i in range(len(list_of_events))]
 

for event in list_of_events:
    for k in range(len(list_of_events)):
        count[str(event)][k] = data_train[ (data_train['event'] == event) & (data_train['next_task'] == list_of_events[k]) ].count()['event']
        
        #if df_new[ df_new['event'] == df_new['next_task'] ]:
            
count

{'A_SUBMITTED': [0,
  10291,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'A_PARTLYSUBMITTED': [0,
  3886,
  3757,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2648,
  0,
  3832,
  0,
  0,
  54],
 'A_PREACCEPTED': [0,
  0,
  5776,
  5776,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'W_Completeren aanvraag': [0,
  0,
  0,
  12770,
  4017,
  22,
  12,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  869,
  908,
  0,
  0,
  0,
  3],
 'A_ACCEPTED': [0,
  0,
  0,
  53,
  0,
  2295,
  1615,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  19,
  35,
  0,
  0,
  0,
  0],
 'O_SELECTED': [0,
  0,
  0,
  0,
  0,
  1,
  2317,
  2463,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  789,
  0,
  0,
  0,
  0,
  0,
  0],
 'A_FINALIZED': [0,
  0,
  0,
  0,
  0,
  1627,
  0,
  2317,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'O_CRE

In [13]:
#The main part of the naive predictor
#Takes the above dictionary and finds the most frequenly occuring consecutive
#event after each event
#Basically, it is our trained model that can be mapped to the original dataset
count2 = count
index = {}
for event in list_of_events:
    max = 0
    for k in range( len(list_of_events)-1 ):
        if count2[event][k] > max:
            max = count2[event][k]
            #print(max)
            index2 = k;
    index[event] = index2
index

{'A_SUBMITTED': 1,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 5,
 'O_SELECTED': 7,
 'A_FINALIZED': 7,
 'O_CREATED': 8,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 15,
 'A_APPROVED': 15,
 'O_ACCEPTED': 12,
 'A_ACTIVATED': 12,
 'O_CANCELLED': 5,
 'A_DECLINED': 20,
 'A_CANCELLED': 16,
 'W_Afhandelen leads': 2,
 'O_DECLINED': 17,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 17}

In [14]:
#Turns the above into an event to event dictionary 

prediction = {}
for event in list_of_events:
    prediction[event] = list_of_events[index[event]]
    
prediction

{'A_SUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PARTLYSUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED': 'A_PREACCEPTED',
 'W_Completeren aanvraag': 'W_Completeren aanvraag',
 'A_ACCEPTED': 'O_SELECTED',
 'O_SELECTED': 'O_CREATED',
 'A_FINALIZED': 'O_CREATED',
 'O_CREATED': 'O_SENT',
 'O_SENT': 'O_SENT',
 'W_Nabellen offertes': 'W_Nabellen offertes',
 'O_SENT_BACK': 'O_SENT_BACK',
 'W_Valideren aanvraag': 'W_Valideren aanvraag',
 'A_REGISTERED': 'A_ACTIVATED',
 'A_APPROVED': 'A_ACTIVATED',
 'O_ACCEPTED': 'A_REGISTERED',
 'A_ACTIVATED': 'A_REGISTERED',
 'O_CANCELLED': 'O_SELECTED',
 'A_DECLINED': 'O_DECLINED',
 'A_CANCELLED': 'O_CANCELLED',
 'W_Afhandelen leads': 'A_PREACCEPTED',
 'O_DECLINED': 'A_DECLINED',
 'W_Nabellen incomplete dossiers': 'W_Nabellen incomplete dossiers',
 'W_Beoordelen fraude': 'A_DECLINED'}

In [15]:
#Adds predicted event column
df_predicted_event = data_test.copy()
df_predicted_event["predicted_event"] = ["" for i in df_predicted_event["event"]]
df_predicted_event['predicted_event'] = df_predicted_event["event"].map(prediction)
df_predicted_event.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,predicted_event
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,O_DECLINED
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,None,A_DECLINED
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,W_Afhandelen leads,A_PARTLYSUBMITTED
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,W_Afhandelen leads,A_PREACCEPTED
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,A_PREACCEPTED
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,A_PREACCEPTED
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,W_Completeren aanvraag,A_PREACCEPTED
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,W_Completeren aanvraag


In [16]:
df_new3 = df_predicted_event.copy()
df_new3["next_task"] = df_new3["next_task"].map(event_to_num)
df_new3["predicted_event"] = df_new3["predicted_event"].map(event_to_num)
df_new3.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,predicted_event
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,20.0,20
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,NaN,17
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,1.0,1
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,1.0,1
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,19.0,1


In [17]:
y_true = df_new3["next_task"].to_numpy().astype(int)
y_pred = df_new3["predicted_event"].to_numpy().astype(int)
#y_true[-1] = 0
y_true, y_pred

(array([         20, -2147483648,           1, ...,          19,
                 17, -2147483648]),
 array([20, 17,  1, ...,  1,  2, 20]))

In [20]:
f1_score(y_true, y_pred, average='micro')

0.5565424723575958

Naive time predictor

In [28]:
df_predicted_time = data_train.copy()
df_predicted_time["duration"] = pd.to_datetime(df_predicted_time["completeTime"]) - pd.to_datetime(df_predicted_time["startTime"])

#Sums up count for each event and the time each event takes
events_count = df_predicted_time.groupby("event")['duration'].agg('count')
event_duration_sum = df_predicted_time.groupby("event")['duration'].agg('sum')

#Computes average duration per event (basically our trained data that can be mapped onto test data)
duration_per_event = event_duration_sum / events_count 

In [29]:
#Final cleanup
df_predicted = df_predicted_event.copy()
df_predicted["predicted_duration"] = df_predicted['event'].map(duration_per_event)
df_predicted = df_predicted.drop(['next_task'], axis=1)
df_predicted.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,predicted_event,predicted_duration
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,00:00:00
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,A_DECLINED,00:00:00
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:15:26.140431
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:15:26.140431
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:00:00
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:00:00
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,00:10:28.478056


In [30]:
#Adding predicted start time of next event
df_predicted_final = df_predicted.copy()
df_predicted_final['predicted_time'] = df_predicted_final["predicted_duration"] + pd.to_datetime(df_predicted_final["startTime"])
df_predicted_final.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,predicted_event,predicted_duration,predicted_time
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,00:00:00,2012-02-20 12:11:35.191000000
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,A_DECLINED,00:00:00,2012-02-20 12:11:35.191000000
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00,2012-02-01 21:51:59.170000000
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00,2012-02-01 21:51:59.265000000
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,00:00:00,2012-02-01 21:51:59.265000000
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:15:26.140431,2012-02-02 09:28:39.224431102
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:15:26.140431,2012-02-02 10:10:26.749431102
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:00:00,2012-02-02 09:59:15.677000000
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,00:00:00,2012-02-02 09:59:15.677000000
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,00:10:28.478056,2012-02-02 11:48:24.950056566


Linear regression

In [23]:
data_iterrated_two = data_iterrated["event"].copy()
data_iterrated_two

0                    A_SUBMITTED
1              A_PARTLYSUBMITTED
2                  A_PREACCEPTED
3                  A_PREACCEPTED
4         W_Completeren aanvraag
                   ...          
190822               A_SUBMITTED
190823         A_PARTLYSUBMITTED
190824         A_PARTLYSUBMITTED
190825        W_Afhandelen leads
190826                A_DECLINED
Name: event, Length: 190827, dtype: object

In [24]:
for i in range(len(data_iterrated_two)):
    data_iterrated_two[i] = event_to_num[data_iterrated_two[i]]

In [25]:
event_to_num

{'A_SUBMITTED': 0,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 4,
 'O_SELECTED': 5,
 'A_FINALIZED': 6,
 'O_CREATED': 7,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 12,
 'A_APPROVED': 13,
 'O_ACCEPTED': 14,
 'A_ACTIVATED': 15,
 'O_CANCELLED': 16,
 'A_DECLINED': 17,
 'A_CANCELLED': 18,
 'W_Afhandelen leads': 19,
 'O_DECLINED': 20,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 22,
 'None': 23}

In [26]:
next_task = data_iterrated["next_task"].copy()
for i in range(len(next_task)):
    #print(next_task[i])
    try:
        next_task[i] = event_to_num[next_task[i]]
    except:
        next_task[i] = 23

In [31]:
#data_iterrated_two["next_task"] = next_task
data_to_lr = data_iterrated.copy()
data_to_lr["event"] = data_iterrated_two
data_to_lr["next_task"] = next_task
data_to_lr

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4
...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17


In [32]:
x = data_to_lr[["event"]]
y = data_to_lr["next_task"]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
naive_predictor = LinearRegression()
naive_predictor.fit(X_train, y_train)
naive_predictor.score(X_test, y_test)
prediction = naive_predictor.predict(data_to_lr[["event"]]).round()

In [34]:
result_data = data_to_lr.copy()
result_data["prediction"] = prediction
result_data["prediction"] = result_data["prediction"].astype(int)

In [35]:
result_data

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1,3
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2,4
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2,5
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3,5
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4,6
...,...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1,3
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1,4
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19,4
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17,18


In [36]:
num_to_event = {v: k for k, v in event_to_num.items()}
num_to_event

{0: 'A_SUBMITTED',
 1: 'A_PARTLYSUBMITTED',
 2: 'A_PREACCEPTED',
 3: 'W_Completeren aanvraag',
 4: 'A_ACCEPTED',
 5: 'O_SELECTED',
 6: 'A_FINALIZED',
 7: 'O_CREATED',
 8: 'O_SENT',
 9: 'W_Nabellen offertes',
 10: 'O_SENT_BACK',
 11: 'W_Valideren aanvraag',
 12: 'A_REGISTERED',
 13: 'A_APPROVED',
 14: 'O_ACCEPTED',
 15: 'A_ACTIVATED',
 16: 'O_CANCELLED',
 17: 'A_DECLINED',
 18: 'A_CANCELLED',
 19: 'W_Afhandelen leads',
 20: 'O_DECLINED',
 21: 'W_Nabellen incomplete dossiers',
 22: 'W_Beoordelen fraude',
 23: 'None'}

In [37]:
predicted_event_str = result_data["prediction"].copy()
for i in range(len(predicted_event_str)):
    predicted_event_str[i] = num_to_event[predicted_event_str[i]]

In [38]:
predicted_event_str

0         W_Completeren aanvraag
1                     A_ACCEPTED
2                     O_SELECTED
3                     O_SELECTED
4                    A_FINALIZED
                   ...          
190822    W_Completeren aanvraag
190823                A_ACCEPTED
190824                A_ACCEPTED
190825               A_CANCELLED
190826                A_DECLINED
Name: prediction, Length: 190827, dtype: object

In [39]:
result_data_str = data_iterrated.copy()
result_data_str["prediction"] = predicted_event_str

In [40]:
result_data_str["duration"] = pd.to_datetime(result_data_str["completeTime"]) - pd.to_datetime(result_data_str["startTime"])
result_data_str.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,00:08:27.480000


In [41]:

# train = result_data_str[result_data_str.index <= len(result_data_str) / 1.2]
# y = train['duration']
# x = train[]
# ARMAmodel = SARIMAX(y, order = (1, 0, 1))


# test = result_data_str[result_data_str.index > len(result_data_str) / 1.2]

# ARMAmodel = ARMAmodel.fit()

# y_pred = ARMAmodel.get_forecast(len(test.index))
# y_pred_df = y_pred.conf_int()#alpha = 0.05) 
# y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
# y_pred_df.index = test.index
# y_pred_out = y_pred_df["Predictions"] 

events_count = result_data_str.groupby("event")['duration'].agg('count')
event_duration_sum = result_data_str.groupby("event")['duration'].agg('sum')

duration_per_event = event_duration_sum / events_count

# for i in range(len(result_data_str)):
#     print(result_data_str.at[i])# = duration_per_event[result_data_str[i]["event"]]

@jit(parallel = True)
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    # idx = 0
    for idx,_ in enumerate(event):
        # print(idx)       
        res[idx] = dict[event[idx]]
        # res[idx] = event[idx + 1]  if (case[idx + 1] != case[idx])
        # else:
        #     res[idx] = 0

        # idx+=1
    return res

result_data_str["duration_prediction"] = make_prediction(result_data_str['event'].values, duration_per_event)

<ipython-input-41-75b75de866ef>:25: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': cannot determine Numba type of <class 'type'>

File "<ipython-input-41-75b75de866ef>", line 27:
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    ^

  @jit(parallel = True)
<ipython-input-41-75b75de866ef>:25: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': cannot determine Numba type of <class 'type'>

File "<ipython-input-41-75b75de866ef>", line 27:
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    ^

  @jit(parallel = True)
C:\ProgramData\Anaconda3\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "make_prediction" was compiled in object mode without forc

In [42]:
result_data_str.head(5)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration,duration_prediction
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,00:00:00,00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,00:00:00,00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,00:00:00,00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,00:00:00,00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,00:08:27.480000,00:10:00.791783


More advanced algorithms (from Hristo)

In [ ]:
# VARMAX example
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from skforecast.ForecasterAutoreg import ForecasterAutoreg # pip install skforecast
from matplotlib import pyplot

In [ ]:
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)][:100]

df_filtered=df_filtered.dropna() #Check later if dropna messes with the dataset through "None" tasks being interpreted as NaN
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

In [ ]:
#OLS regression
import statsmodels.api as sm
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)]


df_filtered=df_filtered.dropna()
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

y = np.array(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = np.array(df_filtered["event"].values, dtype=float)
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
Xnew = np.array(df_prediction["event"].values, dtype=float)
ynewpred = olsres.predict(Xnew) 
df_prediction["time_OLS"] = pd.to_datetime(ynewpred)

In [ ]:
# Lasso Regression
from sklearn import linear_model

y = np.floor(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = df_filtered[["event", "case"]]

lassoReg = linear_model.Lasso()
lassoReg.fit(X, y)

Xnew = np.array(df_prediction[["event", "case"]].values, dtype=float)
df_prediction["time_Lasso"] = pd.to_datetime(lassoReg.predict(Xnew))

Random Fores and Decision Tree for event

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Prep data
result_data.sort_values(by=["case", "event"], inplace=True)
result_data_copy = result_data.copy()
result_data_copy['startTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['startTime']))
result_data_copy['completeTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['completeTime']))
result_data_copy['REG_DATE'] = pd.to_numeric(pd.to_datetime(result_data_copy['REG_DATE']))
result_data_copy.drop(columns='prediction', inplace=True)

In [ ]:
# Split training and test and ensure floating point numbers
train_df, test_df = train_test_split(result_data_copy)

X_train = train_df.drop(['next_task'], axis=1)
Y_train = train_df['next_task']
X_train = X_train.values.astype("float32")
Y_train = Y_train.values.astype("float32")

X_test = test_df.drop(['next_task'], axis=1)
Y_test = test_df['next_task']
X_test = X_test.values.astype("float32")
Y_test = Y_test.values.astype("float32")

In [ ]:
# Fitting and prediction 
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()

DT_fit = DT.fit(X_train, Y_train)
RF_fit = RF.fit(X_train, Y_train)

DT_pred = DT_fit.predict(X_test)
RF_pred = RF_fit.predict(X_test)

In [ ]:
# Accuracy
DT_pred = np.round(DT_pred)
RF_pred = np.round(RF_pred)
print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))
print("Random Forests is %f percent accurate" % (accuracy_score(RF_pred, Y_test)*100))

In [ ]:
# Add to df
test_df["event_DT"] = DT_pred
test_df["event_RF"] = RF_pred

Forecasting ? (from another draft file)

In [ ]:
df_filtered = result_data_str.copy()

df_filtered=df_filtered.dropna() #Check later if dropna messes with the dataset through "None" tasks being interpreted as NaN
# df_prediction=df_prediction.dropna()
df_filtered['duration'] = pd.to_timedelta(df_filtered["duration"])
# df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])


df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

le_event = preprocessing.LabelEncoder()
# le_case = preprocessing.LabelEncoder()
startTimes = df_filtered["event"]#,"case"]]
le_event.fit(startTimes)
# le_case.fit(startTimes["case"])

startTimes = le_event.transform(startTimes)
# startTimes["case"] = le_case.transform(startTimes["case"])
# startTimes["startTime"] = pd.to_numeric(startTimes["startTime"])

In [ ]:
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler()
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)    

def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model    

# supervised = timeseries_to_supervised(startTimesX = series.values
# X = startTimes#.values
# X = X.reshape(len(X), 1)
# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = scaler.fit(X)
# scaled_X = scaler.transform(X)
# scaled_X

# # invert transform
# inverted_X = scaler.inverse_transform(scaled_X)

# inverted_series = Series(inverted_X[:, 0])

# inverted_series

X = startTimes

# train, test = train_test_split(X)


# history = [x for x in train]
# predictions = list()
# for i in range(len(test)):

# rmse = sqrt(mean_squared_error(test, predictions))
# print('RMSE: %.3f' % rmse)

diff_values = difference(X, 1)

supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
train, test = train_test_split(supervised_values)
# train


scaler, train_scaled, test_scaled = scale(train, test)


# fit the model
lstm_model = fit_lstm(train_scaled, 1, 1, 4)



# differenced


In [ ]:
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)


In [ ]:
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(startTimes, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = startTimes[len(train) + i + 1]
	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

Mean absolute error + ? (from another draft file)

In [ ]:
df_result = result_data.sort_values(by=["case", "event"], inplace=True)

df_result["duration"] = pd.to_numeric(pd.to_datetime(df_result["completeTime"]) - pd.to_datetime(df_result["startTime"]))
df_result['REG_DATE'] = pd.to_numeric(df_result['REG_DATE'])
df_result['completeTime'] = pd.to_numeric(pd.to_datetime(df_result['completeTime']))
df_result['startTime'] = pd.to_numeric(pd.to_datetime(df_result['startTime']))

df_filtered = df_result[df_result["case"] < len(df_result)]
df_prediction = df_result[df_result["case"] >= len(df_result)]



df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

listVal = df_filtered
listVal


listValSelected = listVal[['case', 'event', 'startTime', 'completeTime', 'AMOUNT_REQ', 'REG_DATE']]
listValSelected_prediction = df_prediction[['case', 'event', 'startTime', 'completeTime', 'AMOUNT_REQ', 'REG_DATE']]
listValSelected_prediction = listValSelected_prediction.values.astype('float32')
listValDuration_prediction = df_prediction['duration']
listValDuration_prediction = listValDuration_prediction.values.astype('float32')
listValDuration = listVal['duration'].astype('float32')

listValSelected = listValSelected.values.astype('float32')

In [ ]:
# choose a number of time steps
n_steps = len(listValSelected[0])
# split into samples


n_features = 1
X = listValSelected.reshape((listValSelected.shape[0], listValSelected.shape[1], n_features))
y = listValDuration

# define model
model = Sequential()
model.add(LSTM(50, input_shape=(n_steps, n_features), return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(50, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.5))
# model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=20, verbose=2, workers=-1)

# demonstrate prediction
x_input = listValSelected_prediction[:10]

x_input = x_input.reshape((x_input.shape[0], n_steps, n_features))

In [ ]:
yhat = model.predict(x_input, verbose=2)

mean_absolute_error(listValDuration_prediction[:50], yhat[0][:50])

Random forest event prediction (also from different draft file, possibly outdated)

In [ ]:
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)][:100]


df_filtered=df_filtered.dropna()
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])


df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

listVal = df_filtered[["event", "startTime"]]
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(),
                lags = 6
             )


forecaster.fit(y=listVal["event"])

steps = 10
predictions = forecaster.predict(steps=steps)
print(predictions)
# dictionary_events = df_filtered["startTime"]#, dtype=float)[0:18]# np.asarray(
# random_forest = RandomForestClassifier(n_estimators=50,
#                                        max_depth=10, random_state=1)
# random_forest.fit([dictionary_event_startTime[0]], [dictionary_events[0]])
# rf_prediction = random_forest.predict()
# rfe = RFE(estimator=RandomForestRegressor())
# fit = rfe.fit([dictionary_event_startTime[:5]], dictionary_events[:5])
# y_pred = fit.predict([df_prediction["startTime"][:5]])

# y_pred[0]

# dictionary_events
# for i in :

#     data_exog = [x + random() for x in range(100)]
# # fit model
# model = VARMAX(data, exog=data_exog, order=(1, 1))
